In [10]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [11]:
from langchain_community.document_loaders import (
                                                  PyMuPDFLoader,
                                                  NewsURLLoader,
                                                  YoutubeLoader,
                                                  )
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
import pymupdf

import os
import torch
import re
from PIL import Image
from natsort import natsorted
import shutil
from transformers import pipeline
from src.utils import *
from src.data_processing import MathLatexRecovery

In [12]:
device = take_device()

Selected device: cuda


In [13]:
data = config_parser(data_config_path = 'config/data_config.yaml')

In [14]:
pdf_path = "data/web_data/introduction to algo4 new.pdf"
loader = PyMuPDFLoader(pdf_path)
ori_text = loader.load()
ori_text, duplicate_count = remove_duplicate_documents(ori_text[1060:1067])
print(len(ori_text))

7


In [15]:
ori_text[:]

[Document(page_content='Problems for Chapter 33 \n1039 \nwhere \x03 \n.0/ is some value between x \n.0/ and x \n\ue003 . If the approximation in equa- \ntion (33.34) holds for x \n.0/ , it also holds for any point closer to x \n\ue003 . \nb. Assume that the function f has exactly one point x \n\ue003 for which f .x \n\ue003 / D 0. Let \n\x05 \n.t/ D \nˇ \nˇ \nx \n.t/ \ue003 x \n\ue003 ˇ \nˇ \n. Using the Taylor expansion in equation (33.34), show that \n\x05 \n.t C1/ D \nˇ \nˇ \nf 00 .\x03 \n.t/ / \nˇ \nˇ \n2 jf 0 .\x03 \n.t/ /j \x05 \n.t/ ; \nwhere \x03 \n.t/ is some value between x \n.t/ and x \n\ue003 . \nc. If \nˇ \nˇ \nf 00 .\x03 \n.t/ / \nˇ \nˇ \n2 jf 0 .\x03 \n.t/ /j හ c \nfor some constant c and \x05 \n.0/ < 1, then we say that the function f has quadratic \nconvergence, since the error decreases quadratically. Assuming that f has qua- \ndratic convergence, how many iterations are needed to ûnd a root of f .x/ to an \naccuracy of ı \n? Your answer should include ı \n. \nd. Supp

In [16]:
text_list = [txt.page_content for txt in ori_text]

In [17]:
# Use a pipeline as a high-level helper

pipe = pipeline("text-classification", model="CrissWang/bert-math", torch_dtype=torch.float16, device = take_device())

Selected device: cuda


In [18]:
outputs = pipe(text_list, batch_size=8, truncation="only_first",)
print(outputs[:5])

[{'label': 'Math', 'score': 0.66357421875}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}, {'label': 'Not Math', 'score': 1.0}]


In [19]:
# for input, output in zip(text_list, outputs):
#     print(f"output: {output['label']} - Input: {input}")
#     print("---------------")

In [20]:
math_indices = [index for index, output in enumerate(outputs) if output['label'].lower() == "math"]

# Print the list of indices where the label is "math"
print(math_indices[:5])
print(f"There are {len(math_indices)} / {len(ori_text)} pages has math")

[0]
There are 1 / 7 pages has math


In [27]:
def save_math_doc_image( math_indices, pdf_path, tmp_path="data/tmp"):
    doc = pymupdf.open(pdf_path)  # open document
    os.makedirs(tmp_path, exist_ok=True)
    image_paths = []
    for page in doc:  # iterate through the pages
        if not (page.number in math_indices):
            continue
        pix = page.get_pixmap()  # render page to an image
        save_path = f"{tmp_path}/%i.png" % page.number
        pix.save(save_path)  # store image as a PNG
        image_paths.append(save_path)
    return image_paths

tmp_path="data/tmp"
image_list = save_math_doc_image(math_indices, pdf_path,tmp_path)

In [22]:
nougat_processor = pipeline("image-to-text", model="facebook/nougat-base",  device = take_device(), max_new_tokens=1024 )

Selected device: cuda


In [23]:
image_list[:5]

['data/tmp/0.png']

In [28]:
outputs = nougat_processor(image_list, batch_size=8,)

In [29]:
for i, index in enumerate(math_indices):
    for item in outputs[i]:
        item['index'] = index

In [30]:
outputs[:5]

[[{'generated_text': "where \\(\\gamma^{(0)}\\) is some value between \\(x^{(0)}\\) and \\(x^{\\star}\\). If the approximation in equation (33.34) holds for \\(x^{(0)}\\), it also holds for any point closer to \\(x^{\\star}\\).\n\n_b._ Assume that the function \\(f\\) has exactly one point \\(x^{\\star}\\) for which \\(f(x^{\\star})=0\\). Let \\(\\epsilon^{(t)}=\\left|x^{(t)}-x^{\\star}\\right|\\). Using the Taylor expansion in equation (33.34), show that\n\n\\[\\epsilon^{(t+1)}=\\frac{\\left|f^{\\prime\\prime}(\\gamma^{(t)})\\right|}{2\\left|f^{ \\prime}(\\gamma^{(t)})\\right|}\\epsilon^{(t)}\\ ,\\]\n\nwhere \\(\\gamma^{(t)}\\) is some value between \\(x^{(t)}\\) and \\(x^{\\star}\\).\n\n_c._ If\n\n\\[\\frac{\\left|f^{\\prime\\prime}(\\gamma^{(t)})\\right|}{2\\left|f^{\\prime}(\\gamma^{( t)})\\right|}\\leq c\\]\n\n for some constant \\(c\\) and \\(\\epsilon^{(0)}<1\\), then we say that the function \\(f\\) has _quadratic convergence_, since the error decreases quadratically. Assuming 

In [76]:
remove_and_recreate_folder(tmp_path)

In [34]:
for i, item in enumerate(outputs):
    idx = int(item[0]['index'])
    ori_text[idx].page_content = outputs[i][0]["generated_text"]

In [35]:
ori_text = combine_short_doc(ori_text, 100)

In [36]:
ori_text[:5]

[Document(page_content="where \\(\\gamma^{(0)}\\) is some value between \\(x^{(0)}\\) and \\(x^{\\star}\\). If the approximation in equation (33.34) holds for \\(x^{(0)}\\), it also holds for any point closer to \\(x^{\\star}\\).\n\n_b._ Assume that the function \\(f\\) has exactly one point \\(x^{\\star}\\) for which \\(f(x^{\\star})=0\\). Let \\(\\epsilon^{(t)}=\\left|x^{(t)}-x^{\\star}\\right|\\). Using the Taylor expansion in equation (33.34), show that\n\n\\[\\epsilon^{(t+1)}=\\frac{\\left|f^{\\prime\\prime}(\\gamma^{(t)})\\right|}{2\\left|f^{ \\prime}(\\gamma^{(t)})\\right|}\\epsilon^{(t)}\\ ,\\]\n\nwhere \\(\\gamma^{(t)}\\) is some value between \\(x^{(t)}\\) and \\(x^{\\star}\\).\n\n_c._ If\n\n\\[\\frac{\\left|f^{\\prime\\prime}(\\gamma^{(t)})\\right|}{2\\left|f^{\\prime}(\\gamma^{( t)})\\right|}\\leq c\\]\n\n for some constant \\(c\\) and \\(\\epsilon^{(0)}<1\\), then we say that the function \\(f\\) has _quadratic convergence_, since the error decreases quadratically. Assumin

In [ ]:
post_processor = MathLatexRecovery()
outputs = post_processor.recover_math(ori_text)

In [28]:
outputs[30:39]

[Document(page_content='Graph application\nNguy¹n An Kh÷ìng,\nTr¦n Tu§n Anh, Nguy¹n\nTi¸n Thinh\n4.14\nExercise\nFind all the cut vertices, cut edges of the graphs\na) Cn, where n ≥3\nb) Wn where n ≥3\nc) Km,n where m ≥2, n ≥2\n', metadata={'source': 'data/web_data/Graph application 2.pdf', 'file_path': 'data/web_data/Graph application 2.pdf', 'page': 32, 'total_pages': 284, 'format': 'PDF 1.5', 'title': '', 'author': 'Nguyn An Khng, Trn Tun Anh, Nguyn Tin Thinh', 'subject': 'Lecturer Mathematical Foundation For Computer Science', 'keywords': '', 'creator': 'LaTeX with Beamer class', 'producer': 'MiKTeX pdfTeX-1.40.21', 'creationDate': "D:20230917205231+07'00'", 'modDate': "D:20230917205231+07'00'", 'trapped': ''}),
 Document(page_content='Graph application\nNguy¹n An Kh÷ìng,\nTr¦n Tu§n Anh, Nguy¹n\nTi¸n Thinh\n4.15\nExercise\nFor each of these graphs, find κ(G), λ(G)\na)\nb)\n', metadata={'source': 'data/web_data/Graph application 2.pdf', 'file_path': 'data/web_data/Graph application 